In [21]:
postsDF = spark.readStream.format("socket").option("host", "localhost").option("port", "44444").load() 

2024-09-24 10:01:17,519 WARN sources.TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [22]:
from pyspark.sql.functions import split

aPostDF = (postsDF
    .withColumn("id", split(postsDF.value, ",")[0].cast("integer"))
    .withColumn("author_id", split(postsDF.value, ",")[1].cast("integer"))
    .withColumn("title", split(postsDF.value, ",")[2])
    .withColumn("description", split(postsDF.value, ",")[3])
    .withColumn("content", split(postsDF.value, ",")[4])
    .withColumn("date", split(postsDF.value, ",")[5])
)


In [23]:
myOutDF = aPostDF.select("author_id",aPostDF.title[0:10].alias("Post_Title"),"date") 

In [26]:
myStream = (
    myOutDF.writeStream
    .format("console")
    .option("truncate", "false")
    .outputMode("append")
    .trigger(processingTime="2 seconds")
    .start()
)

myStream.awaitTermination()


2024-09-24 10:13:13,505 WARN streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a38a9a0a-2d69-4701-8a16-7285201e3203. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
2024-09-24 10:13:14,667 ERROR streaming.MicroBatchExecution: Query [id = 649948e8-8903-48c7-aed2-4c74ad0d5022, runId = 119f4514-ee99-4e48-afae-20ea817a9d49] terminated with error
java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:476)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:218)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:200)
	at java.net.SocksSocketImpl.connect(SocksSock

StreamingQueryException: Connection refused (Connection refused)
=== Streaming Query ===
Identifier: [id = 649948e8-8903-48c7-aed2-4c74ad0d5022, runId = 119f4514-ee99-4e48-afae-20ea817a9d49]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 44444]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource ConsoleWriter[numRows=20, truncate=false]
+- Project [author_id#162, substring(title#166, 0, 10) AS Post_Title#192, date#184]
   +- Project [value#157, id#159, author_id#162, title#166, description#171, content#177, split(value#157, ,, -1)[5] AS date#184]
      +- Project [value#157, id#159, author_id#162, title#166, description#171, split(value#157, ,, -1)[4] AS content#177]
         +- Project [value#157, id#159, author_id#162, title#166, split(value#157, ,, -1)[3] AS description#171]
            +- Project [value#157, id#159, author_id#162, split(value#157, ,, -1)[2] AS title#166]
               +- Proje